In [9]:
!pip install pyspark==3.0.3

     |████████████████████████████████| 209.1 MB 109 kB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 198 kB 1.0 MB/s            
  Created wheel for pyspark: filename=pyspark-3.0.3-py2.py3-none-any.whl size=209435970 sha256=816d986eaad7eb75ec35f8c5acd5021fb45df423372af290f447081609b82356
  Stored in directory: /home/jovyan/.cache/pip/wheels/65/ff/a2/0e26ceacea69c69610bbbd569678580b54be2e6e8b88e0eb9a
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.2
    Uninstalling py4j-0.10.9.2:
      Successfully uninstalled py4j-0.10.9.2
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.2.0
    Can't uninstall 'pyspark'. No files were found to uninstall.


In [12]:
pyspark.__version__

'3.2.0'

# test kafka installation

In [2]:
from kafka.admin import KafkaAdminClient, NewTopic


admin_client = KafkaAdminClient(
    bootstrap_servers="kafka:9092",
    api_version=(0, 10),
    client_id='test'
)

topic_list = []
topic_list.append(NewTopic(name="example_topic", num_partitions=1, replication_factor=1))
admin_client.create_topics(new_topics=topic_list, validate_only=False)

CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='example_topic', error_code=0, error_message=None)])

In [3]:
from kafka import KafkaProducer

producer = KafkaProducer(
    bootstrap_servers='kafka:9092')

producer.send('example_topic', b'test message', b'key')
producer.flush()
producer.close()

# test spark installation 

In [4]:
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession.builder \
    .master('local[*]') \
    .config('spark.jars', '../src/drivers/aerospike-spark-assembly-3.2.0.jar') \
    .getOrCreate()

In [10]:
spark.stop()

In [4]:
df = spark.read \
    .format('parquet') \
    .load('../data/raw_data/agg_usage.parquet') \
    .limit(1000) \
    .cache()

In [5]:
df.head(10)

[Row(actual_call_dur_sec=115.0, call_direction_ind='1', call_type_key='V', charge_amt_rur=0.0, connection_type_key='3', location_type_key='2', network_type='G', num_of_call=1.0, parent_operator_code_b='136', roaming_type_key='H', rounded_data_volume=1.817427842689825, time_key='P20171025', client_id='0728799'),
 Row(actual_call_dur_sec=122.0, call_direction_ind='2', call_type_key='V', charge_amt_rur=0.0, connection_type_key='3', location_type_key='2', network_type='G', num_of_call=1.0, parent_operator_code_b='130', roaming_type_key='H', rounded_data_volume=1.817427842689825, time_key='P20171025', client_id='0728799'),
 Row(actual_call_dur_sec=48278.0, call_direction_ind='3', call_type_key='G', charge_amt_rur=0.0, connection_type_key='X', location_type_key='X', network_type='L', num_of_call=3.0, parent_operator_code_b='-99', roaming_type_key='H', rounded_data_volume=108065364.52246535, time_key='P20171025', client_id='0728799'),
 Row(actual_call_dur_sec=15150.0, call_direction_ind='3', 

In [24]:
df.groupby('client_id') \
    .sum('rounded_data_volume') \
    .head(1)

[Row(client_id='0728631', sum(rounded_data_volume)=3114051492.7281404)]

# test aerospike connection 

In [1]:
import pyspark
# IP Address or DNS name for one host in your Aerospike cluster
AS_HOST = "127.0.0.1"
# Name of one of your namespaces. Type 'show namespaces' at the aql prompt if you are not sure
AS_NAMESPACE = "test" 
AEROSPIKE_SPARK_JAR_VERSION="3.2.0"
AS_PORT = 3000 # Usually 3000, but change here if not
AS_CONNECTION_STRING = AS_HOST + ":"+ str(AS_PORT)

In [2]:
import os 
AEROSPIKE_JAR_PATH= "../src/drivers/aerospike-spark-assembly-"+AEROSPIKE_SPARK_JAR_VERSION+".jar"
os.environ["PYSPARK_SUBMIT_ARGS"] = '--jars ' + AEROSPIKE_JAR_PATH + ' pyspark-shell'

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
conf=sc._conf.setAll([("aerospike.namespace",AS_NAMESPACE),("aerospike.seedhost",AS_CONNECTION_STRING), ("aerospike.log.level","info")])
sc.stop()

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)


21/11/27 11:59:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [10]:
df.write \
    .mode('overwrite') \
    .format("aerospike")  \
    .option("aerospike.writeset", "test")\
    .option("aerospike.updateByKey", "id") \
    .save()

Py4JJavaError: An error occurred while calling o90.save.
: java.lang.NoClassDefFoundError: org/apache/spark/sql/internal/connector/SupportsStreamingUpdate
	at com.aerospike.spark.sql.sources.v2.AerospikeTable.newWriteBuilder(AerospikeTable.scala:168)
	at org.apache.spark.sql.execution.datasources.v2.V2Writes$.org$apache$spark$sql$execution$datasources$v2$V2Writes$$newWriteBuilder(V2Writes.scala:95)
	at org.apache.spark.sql.execution.datasources.v2.V2Writes$$anonfun$apply$1.applyOrElse(V2Writes.scala:56)
	at org.apache.spark.sql.execution.datasources.v2.V2Writes$$anonfun$apply$1.applyOrElse(V2Writes.scala:38)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$1(TreeNode.scala:481)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:82)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDownWithPruning(TreeNode.scala:481)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.org$apache$spark$sql$catalyst$plans$logi

In [22]:
#Python
#sc is SparkContext, spark is SparkSession object 
#import the package
sc = spark.sparkContext
scala_object= sc._jvm.com.aerospike.spark.PythonUtil
#get java object representing the dataframe
gateway_df=scala_object.aerolookup(dataframe._jdf, keyCol, set, outputSchemaAsJson, namespace)
#convert the java object to python dataframe
aerolookup_df=pyspark.sql.DataFrame(gateway_df, spark._wrapped)

NameError: name 'dataframe' is not defined